<a href="https://colab.research.google.com/github/brainmusic/models/blob/master/BrainMusic_Train_polyphony_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup
Install magenta and fluidsynth, a sequence synthesis.

In [0]:
#@title Install

#@markdown Install magenta and fluidsynth as a synthesizer to listen de audios.
 #@markdown Magenta is compatible with both Python 2 and 3.
 #@markdown This take some time, specially for fluidsynth instalation

!pip install magenta
!apt-get update -qq && apt-get install -qq libfluidsynth1 \
fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

In [0]:
#@title Drive Setup
#@markdown If your training sample is in google drive you need to connect it. 
#@markdown You can also upload the data to a temporary folder but it will be 
#@markdown lost when the session is closed.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#@title Import Dependencies
#@markdown Import libraries from Magenta, Tensorflow and Numpy

from google.colab import files
import numpy as np
import os
import tensorflow as tf
import magenta.music as mm
import magenta
from magenta.scripts import convert_dir_to_note_sequences
from magenta.models.polyphony_rnn import *



# Sample adaptation

Magenta does not work directly with Midi files but with NoteSequences, so the first step to create the sample is to convert the Midi files into Note sequences files and pack them as tfrecord to work with them fast and efficiently.


You need to define the folder with the midi files, as well as the
folder in which the notesequence will be saved. Change the routes from below to your own folders. The output of this step (notesequences)  can be saved directly as a temporary file, e.g. /tmp/notesequences_Happy.tfrecord \ instead of on the drive, since after the next step in which the sample is split it would no longer be necessary.  

Tips for novice: You can see the path of the folder to the left in files and with the mouse button select "copy path". As the route usually contains   "My drive" remember to enter "\\"  in the middel as "My \ drive".

In [0]:
!convert_dir_to_note_sequences \
--input_dir=/content/drive/My\ Drive/brain_music/data/audios/Happy \
--output_file=/content/drive/My\ Drive/brain_music/data/audios/noteSequences/notesequences_Happy.tfrecord \
--log=INFO

Now you are ready to separate your sample between train and test. The percentage you leave in the test for evaluate your model is defined with the eval ratio argument. For example with a eval ratio equal to 10%, the 90% of the sample will be saved in the traing collection, while the remaining 10% will be stored as evaluation sample.

The input for this step must match the one defined as output in the previous step.


In [0]:
#test and train sample split wih 10% ratio
!polyphony_rnn_create_dataset \
--input=/content/drive/My\ Drive/brain_music/data/audios/noteSequences/notesequences_Happy.tfrecord \
--output_dir=/content/drive/My\ Drive/brain_music/RNN/sample/Happy \
--eval_ratio=0.10 \
--config='polyphony'

If the cell has been executed correctly, you have to have two files saved in the output_dir, both in tfrecord format, one with the training sample and one with the eval sample. 


# Model Training
Now you are ready to train your model!

This step can take a long time and depending on how large your database is and the number of layers and their size, you may get a memory error, or lose the connection to the GPU.

We recommend you to start with a small sample and a light model for example a bach size of 64 batch_size=64 and two LSTM rnn layers of 64, "batch_size=64,rnn_layer_sizes=[64,64]" and incorporate more complexity little by little. If you save the checkpoints you can re-launch the training at the point where you left the previous session, this is especially interesting if you lose the web connection or your session closes unexpectedly.  


To train the model you can define the following parameters: 
* **run_dir** is the directory where checkpoints and TensorBoard data will be stored.
* **sequence_example_file** is the TFRecord file with the train sample, the folder  must be the same as the one defined in output_dir in the previous step . 
* **num_training_steps** is an optional parameter for how many update steps to take before exiting the training loop. By default, training will run continuously until manually terminated.
* **hparams** is another optional parameter that specifies the hyperparameters you want to use. By default polyphony rnn model use this configuration 
* **dropout_keep_prob** Dropout is a regularization method to reduce overfitting and improving model performance. The dropout select randomly a % of neurons in the LSTM units thats are probabilistically excluded from activation and weight updates while training the model. 
* **learning_rate** The learning rate controls how quickly or slowly a neural network model learns. This value are ussually between 0.0 and 1.0, a learning rate too small may result in a long training process that could get stuck, whereas a value too large may result in an unstable training process.

By default polyphony_rnn model use this configuration: 
* batch_size=64,
* rnn_layer_sizes=[256, 256, 256]
* dropout_keep_prob=0.5
* learning_rate=0.001

In [0]:
#Train the model!
!polyphony_rnn_train \
--run_dir=/content/drive/My\ Drive/brain_music/RNN/modelos/polyphony/happy/run2 \
--sequence_example_file=/content/drive/My\ Drive/brain_music/RNN/sample/Happy/training_poly_tracks.tfrecord \
--num_training_steps=10000 \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--config='polyphony' \
--num_checkpoints=10


When you consider that the model is sufficiently tuned you can keep it in a bundle file. This allows you to import the trained model at any time and use it to create new sequences. To save it you have to call the same function of the previous step polyphony_rnn_generate, but changing some of the parameterization

*   the run directory has to be the same as in previous step 
*   hparam must also be the same as those defined in the training. 
*   bundle_file is the path where to save the file with the model.mag









In [0]:
#Save your model 
!melody_rnn_generate \
--run_dir=/content/drive/My\ Drive/brain_music/RNN/modelos/polyphony/happy/run2 \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--bundle_file=/content/braimusic_poly_happy_rnn.mag \
--save_generator_bundle

# Sample adaptation


In [0]:
primer_midi=('/content/drive/My Drive/brain_music/data/audios/Happy/temp110__1.mid')
input_sequence=mm.midi_file_to_note_sequence(primer_midi)
start = magenta.music.sequences_lib.extract_subsequence(input_sequence,0,12)

In [0]:
#generar nuevas canciones con el modelo entrenado a partir de inicio midi
!melody_rnn_generate \
--config='attention_rnn' \
--run_dir=/content/drive/My\ Drive/brain_music/RNN/modelos/polyphony \
--output_dir=/content/drive/My\ Drive/brain_music/data/audios/creacionesAI/polyphony_train \
--num_outputs=1 \
--num_steps=600 \
--primer_midi=/content/drive/My\ Drive/brain_music/OmensOfLove.mid \
--condition_on_primer=true \
--temperature=0.9
--inject_primer_during_generation=false



In [0]:
mm.plot_sequence(input_sequence)
primer_midi_new=('/tmp/rnn/generated2/2019-11-26_220959_1.mid')
input_sequence_new=mm.midi_file_to_note_sequence(primer_midi_new)
mm.plot_sequence(input_sequence_new)

In [0]:
#generate a  some new sequences with the train model
!polyphony_rnn_generate \
--config='polyphony_rnn ' \
--bundle_file=/content/braimusic_poly_happy_rnn.mag  \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--output_dir=/content/drive/My\ Drive/brain_music/data/audios/creacionesAI/polyphony_pretrain \
--num_outputs=10 \
--num_steps=600 \
--primer_pitches="[67,64,60]" \
--condition_on_primer=true \
--inject_primer_during_generation=false

In [0]:
!polyphony_rnn_generate \
--config='polyphony_rnn ' \
--bundle_file=/content/polyphony_rnn.mag  \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--output_dir=/content/drive/My\ Drive/brain_music/data/audios/creacionesAI/polyphony_pretrain \
--num_outputs=10 \
--num_steps=600 \
--primer_pitches="[67,64,60]" \
--condition_on_primer=true \
--inject_primer_during_generation=false

In [0]:
#modelo pre-entrenado
from magenta.models.polyphony_rnn import polyphony_sequence_generator
mm.notebook_utils.download_bundle('polyphony_rnn.mag', '/content/')
bundle_polyphony = mm.sequence_generator_bundle.read_bundle_file('/content/polyphony_rnn.mag')

In [0]:
from magenta.models.performance_rnn import *
#modelo pre-entrenado
from magenta.models.performance_rnn import performance_sequence_generator
mm.notebook_utils.download_bundle('performance.mag', '/content/')
bundle_performance = mm.sequence_generator_bundle.read_bundle_file('/content/performance.mag')

mm.notebook_utils.download_bundle('performance_with_dynamics.mag', '/content/')
bundle_performance_with_dynamics = mm.sequence_generator_bundle.read_bundle_file('/content/performance_with_dynamics.mag')


mm.notebook_utils.download_bundle('performance_with_dynamics_and_modulo_encoding.mag', '/content/')
bundle_performance_with_dynamics_and_modulo_encoding = mm.sequence_generator_bundle.read_bundle_file('/content/performance_with_dynamics_and_modulo_encoding.mag')


In [0]:
!performance_rnn_generate \
--config='performance_rnn ' \
--bundle_file=/content/performance.mag  \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--output_dir=/content/drive/My\ Drive/brain_music/data/audios/creacionesAI/performance_pretrain \
--num_outputs=10 \
--num_steps=6000 \
--primer_pitches="[67,64,60]" \
--condition_on_primer=true \
--inject_primer_during_generation=false

W1130 17:02:59.275494 139954035664768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/pipelines/statistics.py:131: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W1130 17:03:00.009083 139954035664768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/note_sequence_io.py:60: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W1130 17:03:00.011415 139954035664768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1130 17:03:00.399406 139954035664768 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta